<a href="https://colab.research.google.com/github/prajwal-suresh13/nlp/blob/main/imdb_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDb Sentiment analysis using DistilBERT

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 31.1 MB/s 
     |████████████████████████████████| 182 kB 101.5 MB/s 
     |████████████████████████████████| 7.6 MB 65.1 MB/s 


In [5]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 14.6 MB/s 
     |████████████████████████████████| 212 kB 84.7 MB/s 
     |████████████████████████████████| 132 kB 78.6 MB/s 
     |████████████████████████████████| 127 kB 88.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# Loading data

We are using IMDb dataset provided by huggingface for Sequence classification

In [6]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
imdb_dataset['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

# Tokenization

In [9]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding

In [10]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [28]:
def preprocess_func(example):
    return tokenizer(example["text"], truncation=True)

In [29]:
tokenized_imdb = imdb_dataset.map(preprocess_func, batched=True)
tokenized_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

Datacollator adds padding to each example in our data such that all the examples in the batch have same length of tokens

In [13]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer) 

# Model and training

In [14]:
imdb_classifier = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bia

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    "/content/gdrive/My Drive/imdb_distillbert",
    num_train_epochs = 2,
    per_device_train_batch_size = 24,
    per_device_eval_batch_size = 36,
    
    evaluation_strategy='epoch'
)

In [32]:
from datasets import load_metric
import numpy as np
metric = load_metric("accuracy")

def compute_metrics(eval_pred):  # custom method to take in logits and calculate accuracy of the eval set
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-32-3970152eba49>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [33]:
trainer = Trainer(
    imdb_classifier,
    training_args,
    train_dataset = tokenized_imdb["train"],
    eval_dataset = tokenized_imdb["test"],
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [34]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 2
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 2084
  Number of trainable parameters = 65783042


Epoch,Training Loss,Validation Loss,Accuracy
1,0.161800,0.197820,0.923720
2,0.064400,0.303844,0.927000


Saving model checkpoint to imdb_distillbert/checkpoint-500
Configuration saved in imdb_distillbert/checkpoint-500/config.json
Model weights saved in imdb_distillbert/checkpoint-500/pytorch_model.bin
tokenizer config file saved in imdb_distillbert/checkpoint-500/tokenizer_config.json
Special tokens file saved in imdb_distillbert/checkpoint-500/special_tokens_map.json
Saving model checkpoint to imdb_distillbert/checkpoint-1000
Configuration saved in imdb_distillbert/checkpoint-1000/config.json
Model weights saved in imdb_distillbert/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in imdb_distillbert/checkpoint-1000/tokenizer_config.json
Special tokens file saved in imdb_distillbert/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore t

TrainOutput(global_step=2084, training_loss=0.11779665123249428, metrics={'train_runtime': 3089.5743, 'train_samples_per_second': 16.183, 'train_steps_per_second': 0.675, 'total_flos': 6616936984752768.0, 'train_loss': 0.11779665123249428, 'epoch': 2.0})

In [38]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 36


{'eval_loss': 0.3038442134857178,
 'eval_accuracy': 0.927,
 'eval_runtime': 403.3561,
 'eval_samples_per_second': 61.98,
 'eval_steps_per_second': 1.723,
 'epoch': 2.0}